In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!git clone https://github.com/smrchcoder/CommentToxicity

fatal: destination path 'CommentToxicity' already exists and is not an empty directory.


In [8]:
#installing all the libariries
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [9]:
df = pd.read_csv('/content/CommentToxicity/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv')

Preprocessing layer


In [10]:
from tensorflow.keras.layers import TextVectorization

In [11]:
# Text Cleaning (example: converting text to lowercase)
X = df['comment_text'].str.lower()

y = df[df.columns[2:]].values
MAX_FEATURES = 200000  # number of words in the vocab

In [12]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
# Stopword Removal
stopwords = set(stopwords.words('english'))
X = X.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
# TextVectorization with preprocessing steps
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=1800,
    output_mode='int',
    # Text Cleaning (example: removing punctuation)
    standardize='lower_and_strip_punctuation',
    # Tokenization (example: splitting on whitespace)
    split='whitespace',
)

In [14]:
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

In [15]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [16]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dropout(0.2))  # Dropout for regularization
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
# Final layer
model.add(Dense(6, activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [ ]:
history = model.fit(train, epochs=10, validation_data=val)

Epoch 1/10
6981/6981 [==============================] - 647s 93ms/step - loss: 0.0222 - val_loss: 0.0193
Epoch 2/10
6981/6981 [==============================] - 622s 89ms/step - loss: 0.0206 - val_loss: 0.0165
Epoch 3/10
6981/6981 [==============================] - 620s 89ms/step - loss: 0.0195 - val_loss: 0.0167
Epoch 4/10
6981/6981 [==============================] - 619s 89ms/step - loss: 0.0184 - val_loss: 0.0150
Epoch 5/10
6981/6981 [==============================] - 621s 89ms/step - loss: 0.0176 - val_loss: 0.0131
Epoch 6/10
6981/6981 [==============================] - 623s 89ms/step - loss: 0.0167 - val_loss: 0.0130
Epoch 7/10
6981/6981 [==============================] - 617s 88ms/step - loss: 0.0154 - val_loss: 0.0116
Epoch 8/10
6981/6981 [==============================] - 623s 89ms/step - loss: 0.0151 - val_loss: 0.0109
Epoch 9/10
6981/6981 [==============================] - 612s 88ms/step - loss: 0.0139 - val_loss: 0.0110
Epoch 10/10
6981/6981 [==============================] 

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

1/1 [==============================] - 0s 57ms/step
Precision: 0.9526968598365784, Recall:0.9494107365608215, Accuracy:0.4653961956501007


In [ ]:
from tensorflow.keras.metrics import Precision, Recall, Accuracy

pre = Precision()
re = Recall()
acc = Accuracy()

for batch in test.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')


1/1 [==============================] - 0s 75ms/step
Precision: 0.9563854336738586, Recall: 0.9512333273887634, Accuracy: 0.3860435485839844


Creating a dilbert based model

In [18]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(64, activation='tanh')))
model.add(Dropout(0.2))  # Dropout for regularization
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
# Final layer
model.add(Dense(6, activation='sigmoid'))

In [19]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [20]:
history = model.fit(train, epochs=10, validation_data=val)

Epoch 1/10
6981/6981 [==============================] - 792s 112ms/step - loss: 0.0658 - val_loss: 0.0452
Epoch 2/10
6981/6981 [==============================] - 666s 95ms/step - loss: 0.0474 - val_loss: 0.0434
Epoch 3/10
6981/6981 [==============================] - 642s 92ms/step - loss: 0.0425 - val_loss: 0.0377
Epoch 4/10
6981/6981 [==============================] - 657s 94ms/step - loss: 0.0391 - val_loss: 0.0327
Epoch 5/10
6981/6981 [==============================] - 657s 94ms/step - loss: 0.0356 - val_loss: 0.0302
Epoch 6/10
6981/6981 [==============================] - 670s 96ms/step - loss: 0.0321 - val_loss: 0.0276
Epoch 7/10
6981/6981 [==============================] - 660s 95ms/step - loss: 0.0300 - val_loss: 0.0256
Epoch 8/10
6981/6981 [==============================] - 658s 94ms/step - loss: 0.0278 - val_loss: 0.0233
Epoch 9/10
6981/6981 [==============================] - 656s 94ms/step - loss: 0.0257 - val_loss: 0.0223
Epoch 10/10
6981/6981 [==============================]

In [21]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 55ms/step


In [22]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision: 0.9218841195106506, Recall: 0.8796761631965637, Accuracy: 0.4914744198322296


In [23]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [24]:
import tensorflow as tf
import gradio as gr

In [25]:
model.save('toxicity.h5')

In [26]:
model = tf.keras.models.load_model('toxicity.h5')

In [30]:
input_str = vectorizer('I hate you!')

In [31]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 0s 52ms/step


In [32]:
res


array([[0.5730448 , 0.00059666, 0.02704595, 0.00364518, 0.05866145,
        0.00515176]], dtype=float32)

In [33]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text
interface = gr.Interface(fn=score_comment,
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')
interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://43c2c6d246ce9b984b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
